In [ ]:
# EV Routing pakcage
from ev_routing.floyd_warshall_profile import FloydWarshallProfile

# Visualization packages
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly"

# Image directory
from pathlib import Path
IMAGEDIR = './FW_profile'
Path(IMAGEDIR).mkdir(parents=True, exist_ok=True)

import os


# Initializing Dijsktra
bl = {'lat': 52.510e0, 'lon': 13.385e0} # Bottom left corner coordinate
tr = {'lat': 52.520e0, 'lon': 13.389e0} # Top right corner coordinate

area = [bl['lat'], bl['lon'], tr['lat'], tr['lon']]


# Initializing bapbox
mapbox_access_token = os.environ['mapbox_access_token']

# Testing mode
testing = False
M = 3000
minus_inf = -.2 * M

In [ ]:
fwp = FloydWarshallProfile(area, M, testing=testing)

latitudes = [v['lat'] for v in fwp.v.values()]
longitudes = [v['lon'] for v in fwp.v.values()]

node_ids = [v['id'] for v in fwp.v.values()]
node_labels = [str(i) for i in range(len(fwp.v))]

In [ ]:
nodes_map = go.Figure()

nodes_map.add_trace( go.Scattermapbox(
    lat = latitudes,
    lon = longitudes,
    mode = 'markers',
    marker = go.scattermapbox.Marker(
        size = 12,
        color = 'rgb(242, 177, 172)',
        opacity=1.0
    ),
    text = node_labels,
    hoverinfo = 'text'
))

nodes_map.update_layout(
    autosize = True,
    hovermode = 'closest',
    showlegend = False,
    mapbox = go.layout.Mapbox(
        accesstoken = mapbox_access_token,
        bearing = 0,
        center = go.layout.mapbox.Center(
            lat = fwp.map_center[0],
            lon = fwp.map_center[1]
        ),
        pitch = 0,
        zoom = 14,
        style = 'light'
    ),
)

selected_nodes = sorted([60, 37, 171, 187, 80, 4, 17, 96, 274, 44])
for n in selected_nodes:
    nodes_map.add_trace(go.Scattermapbox(
        lat=[latitudes[n]], lon=[longitudes[n]], mode = 'markers+text',
        marker=go.scattermapbox.Marker(size=16, color='black'),
        text=str(n), textposition="top center"
    ))

# for e in fwp.e.values():
#     u = fwp.v[e['u']]
#     v = fwp.v[e['v']]
#     c = e['cost']
#     nodes_map.add_trace(go.Scattermapbox(
#         lat=[u['lat'], v['lat']], lon=[u['lon'], v['lon']], text=str(c), mode='lines+markers',
#         marker = go.scattermapbox.Marker(symbol=['marker-stroked', 'marker'])
#     ))

nodes_map.update_layout(margin = {"r": 0, "t": 0, "l": 0, "b": 0})
nodes_map.write_image(IMAGEDIR + '/nodes.png', scale=3)
nodes_map.show()

In [ ]:
import time

start_time = time.time()
history = fwp.run_with_history()
print("%s s" % (time.time() - start_time))

In [ ]:


def plot_gsoc_func(bps, fig, minus_inf, row=None, col=None, color=None):
    if color == None:
        color = 'black'
        
    for bp_id, bp in enumerate(bps):
        x, y, text = [], [], []
        
        x.append(bp[0])
        y.append(bp[1] if bp[1] > float('-inf') else minus_inf)
        
        if bp_id == len(bps) - 1:
            x.append(x[-1])
        else:
            x.append(bps[bp_id + 1][0])
            
        if bp[2] == 1:
            y.append(bp[1] + (x[-1] - bp[0]))
        else:
            y.append(bp[1] if bp[1] > float('-inf') else minus_inf)
            
        text.append(str(bp))
        
        if row is None or col is None:
            fig.add_trace(go.Scatter(x=x, y=y, text=text, mode='lines+markers',
                                    line=dict(width=2, color=color),
                                    marker=dict(color=color, size=5, symbol=['circle', 'circle-open']),
                                    ))
        else:
            fig.add_trace(go.Scatter(x=x, y=y, text=text, mode='lines+markers',
                                    line=dict(width=2, color=color),
                                    marker=dict(color=color, size=5, symbol=['circle', 'circle-open']),
                                    ), row, col)

grid = (len(history[0]), len(history[0][0]))


# Run for a single item in the matrix
# single_fig = go.Figure()
# for i in range(0, len(history), 20):
#     plot_gsoc_func(history[i][251][53], single_fig, minus_inf)
# single_fig.show()

grid_len = len(selected_nodes)
ts = [0] + sorted(selected_nodes) + [len(fwp.v) - 1]

for t in ts:
    print('plotting step ' + str(t) + '...')
    fig = make_subplots(
        grid_len, grid_len, column_widths=[1] * grid_len, row_heights=[1] * grid_len,
        subplot_titles=['(' + str(i) + ', ' + str(j) + ')' for i in selected_nodes for j in selected_nodes]
    )

    for i_id, i in enumerate(selected_nodes):
        for j_id, j in enumerate(selected_nodes):
            plot_gsoc_func(history[t][i][j], fig, minus_inf, row=i_id+1, col=j_id+1)
            fig.update_xaxes(title_text="", range=(-.1 * M, 1.1 * M), row=i_id+1, col=j_id+1)
            fig.update_yaxes(title_text="", range=(minus_inf - .1 * M, 1.1 * M), row=i_id+1, col=j_id+1)

    fig.update_layout(
        title='gSoC functions (step = ' + str(t) + ')',
        width=1200, height=1600,
        showlegend=False
    )


    fig.write_image(IMAGEDIR + '/' + 'history_' + '-'.join([str(x) for x in selected_nodes]) + ('_%03d' % t) + '.png', width=1200, height=1600, scale=3)
#     fig.show()